In [28]:
%%time
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yaml
from datetime import datetime
import time

Wall time: 0 ns


In [29]:
%%time
alltour = 'http://liquipedia.net/dota2/Premier_Tournaments'
htmlall = requests.get(alltour)
htmlall.encoding = 'utf-8'
soup = BeautifulSoup(htmlall.text, "lxml")
divs = soup.find_all('b')
urls = []
for div in divs:
    urls.append(re.search(r'href="(.+?)"', str(div))[1])
df = pd.DataFrame(columns = ['team',
                             'player',
                             'position',
                             'tour',
                             'place',
                             'prize',
                             'start', 
                             'end', 
                             'startelo32',
                             'startelo64',
                             'startglickorating',
                             'startglickomu',
                             'startglickophi',
                             'startglickosigma',
                             'startglicko2rating',
                             'startglicko2mu',
                             'startglicko2phi',
                             'startglicko2sigma',
                             'endelo32',
                             'endelo64',
                             'endglickorating',
                             'endglickomu',
                             'endglickophi',
                             'endglickosigma',
                             'endglicko2rating',
                             'endglicko2mu',
                             'endglicko2phi',
                             'endglicko2sigma'])
df2 = pd.DataFrame(columns = ['tour',
                              'place',
                              'prize',
                              'team'])
i = 0

Wall time: 1.04 s


In [52]:
%%time
for url in urls[67:]:
    try:
        i += 1
        url = 'http://wiki.teamliquid.net' + url
        html = requests.get(url)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, "lxml")
        #find tournament name
        tournamentname = soup.find_all('span', dir="auto")
        tournamentname = tournamentname[0].contents[0]
        #find tournament prizepool
        prizedict = {}
        prizess = soup.find_all('table', class_="table table-bordered prizepooltable collapsed")
        prizes = prizess[0].find_all('tr')
        place = 0
        sameplace = 0
        for p in prizes[1:]:
            if re.search(r'[$]\w+,\w+,\w+|[$]\w+,\w+|[$]\w+', str(p)) is not None:
                prize = re.search(r'[$]\w+,\w+,\w+|[$]\w+,\w+|[$]\w+', str(p))                
                place += 1 + sameplace
                sameplace = 0
                prize = prize[0]
                team = p.find_all('a')[-1].contents[0]
                df2 = df2.append({'tour': tournamentname,
                           'place': place,
                           'prize': prize,
                           'team': team}, ignore_index=True)
                teamlower = re.sub(r'\W+', '', team).lower()
                prizedict[teamlower] = [place, prize]
            else:
                sameplace += 1
                team = p.find_all('a')[-1].contents[0]
                teamlower = re.sub(r'\W+', '', team).lower()
                prizedict[teamlower] = [place, prize]                
                df2 = df2.append({'tour': tournamentname,
                           'place': place,
                           'prize': prize,
                           'team': team}, ignore_index=True).fillna(method='ffill') 
        #find tournament info
        info = soup.find_all('div', text = re.compile(r'\d*-\d*-\d*'), class_="infobox-cell-2")
        startdate = info[0].contents[0]
        enddate = info[1].contents[0]
        #check if tournament is already played
        if datetime.strptime(enddate, '%Y-%m-%d') > datetime.now():
            continue
        startdate2 = startdate[-2:] + startdate[4:8] + startdate[:4]
        enddate2 = enddate[-2:] + enddate[4:8] + enddate[:4]
        #rating of startdate from datdota and turn into list of dict
        urldat = 'http://www.datdota.com/api/ratings?date=' + startdate2
        htmldat = requests.get(urldat)
        htmldat.encoding = 'utf-8'
        elo = BeautifulSoup(htmldat.text, "lxml")
        elodict = yaml.load(elo.text)
        startelolist = elodict['data']   
        #rating of enddate from datdota and turn into list of dict
        urldat = 'http://www.datdota.com/api/ratings?date=' + enddate2
        htmldat = requests.get(urldat)
        htmldat.encoding = 'utf-8'
        elo = BeautifulSoup(htmldat.text, "lxml")
        elodict = yaml.load(elo.text)
        endelolist = elodict['data']
        
        #divs contain all of team info in tournament
        divs = soup.find_all('div', class_="teamcard") 
        
        #for each info of team
        for div in divs:
            #find teamname
            center = div.find('center')
            teamname = re.search(r'title="(.+?)"', str(center))[1]
            teamnamelower = re.sub(r'\W+', '', teamname).lower()
            #init elo32,64
            startelo32 = 'N/A'
            startelo64 = 'N/A'
            startglickorating = 'N/A'
            startglickomu = 'N/A'
            startglickophi = 'N/A'
            startglickosigma = 'N/A'
            startglicko2rating = 'N/A'
            startglicko2mu = 'N/A'
            startglicko2sigma = 'N/A'
            endelo32 = 'N/A'
            endelo64 = 'N/A'
            endglickorating = 'N/A'
            endglickomu = 'N/A'
            endglickophi = 'N/A'
            endglickosigma = 'N/A'
            endglicko2rating = 'N/A'
            endglicko2mu = 'N/A'
            endglicko2phi = 'N/A'
            endglicko2sigma = 'N/A'
            
            #for each dict in list of dict of startelo compare teamname to teamname from liquipedia
            for startelo in startelolist:
                startelolower = re.sub(r'\W+', '', startelo['teamName'].lower())
                if (((startelolower in teamnamelower)
                or (teamnamelower in startelolower))
                and (abs(len(teamnamelower) - len(startelolower)) < 5)):
                    startelo32 = startelo['elo32']['current']
                    startelo64 = startelo['elo64']['current']
                    startglickorating = startelo['glicko']['rating']
                    startglickomu = startelo['glicko']['mu']
                    startglickophi = startelo['glicko']['phi']
                    startglickosigma = startelo['glicko']['sigma']
                    startglicko2rating = startelo['glicko2']['rating']
                    startglicko2mu = startelo['glicko2']['mu']
                    startglicko2phi = startelo['glicko2']['phi']                    
                    startglicko2sigma = startelo['glicko2']['sigma']
            for endelo in endelolist:
                endelolower = re.sub(r'\W+', '', endelo['teamName'].lower())
                if (((endelolower in teamnamelower)
                or (teamnamelower in endelolower))
                and (abs(len(teamnamelower) - len(endelolower)) < 5)):
                    endelo32 = endelo['elo32']['current']
                    endelo64 = endelo['elo64']['current']
                    endglickorating = endelo['glicko']['rating']
                    endglickomu = endelo['glicko']['mu']
                    endglickophi = endelo['glicko']['phi']
                    endglickosigma = endelo['glicko']['sigma']
                    endglicko2rating = endelo['glicko2']['rating']
                    endglicko2mu = endelo['glicko2']['mu']
                    endglicko2phi = endelo['glicko2']['phi']
                    endglicko2sigma = endelo['glicko2']['sigma']

            for table in div.find_all('table', class_="table table-bordered list"):
                for tr in table.find_all('tr'):
                    string = re.search(r'<th>([12345])</th>.+title=".+">(.+)</a>', str(tr))
                    if string is None:
                        continue
                    df = df.append({'team': teamname,
                                    'position': string[1],
                                    'player': string[2],  
                                    'tour': tournamentname, 
                                    'place': prizedict[teamnamelower][0],
                                    'prize': prizedict[teamnamelower][1],
                                    'start': startdate, 
                                    'end': enddate, 
                                    'startelo32': startelo32,
                                    'startelo64': startelo64,
                                    'startglickorating': startglickorating,
                                    'startglickomu': startglickomu,
                                    'startglickophi': startglickophi,
                                    'startglickosigma': startglickosigma,
                                    'startglicko2rating': startglicko2rating,
                                    'startglicko2mu': startglicko2mu,
                                    'startglicko2phi': startglicko2phi,
                                    'startglicko2sigma': startglicko2sigma,
                                    'endelo32': endelo32,
                                    'endelo64': endelo64,
                                    'endglickorating': endglickorating,
                                    'endglickomu': endglickomu,
                                    'endglickophi': endglickophi,
                                    'endglickosigma': endglickosigma,
                                    'endglicko2rating': endglicko2rating,
                                    'endglicko2mu': endglicko2mu,
                                    'endglicko2phi': endglicko2phi,
                                    'endglicko2sigma': endglicko2sigma}, ignore_index=True)  
        time.sleep(3)
    except IndexError:
        continue
        
    except NameError:
        continue
        
    except TypeError:
        continue
        
    except KeyError:
        continue

Wall time: 7min 24s


In [47]:
i

71

In [46]:
df[(df['tour'] == 'The International 2017')].groupby('team').size()

team
Cloud9               5
Digital Chaos        5
Evil Geniuses        5
Execration           5
Fnatic               5
HellRaisers          5
IG Vitality          5
Infamous             5
Invictus Gaming      5
LGD Gaming           5
LGD.Forever Young    5
Newbee               5
OG                   5
TNC Pro Team         5
Team Empire          5
Team Liquid          5
Team Secret          5
Virtus.pro           5
dtype: int64

In [54]:
len(df)

5492

In [58]:
df = df.drop_duplicates()

In [61]:
df.to_csv('premier2.csv', index=False)

In [63]:
df2.to_csv('premier2tourinfo.csv', index=False)

In [20]:
df.head()

,team,player,position,tour,place,prize,start,end,startelo32,startelo64,...,endelo32,endelo64,endglickorating,endglickomu,endglickophi,endglickosigma,endglicko2rating,endglicko2mu,endglicko2phi,endglicko2sigma
0,Infamous,Kotarō Hayama,1,Galaxy Battles II: Emerging Worlds,7,"$20,000",2018-01-19,2018-01-21,1116.27,1172.69,...,1040.73,1061.77,898.102,1686.2,None,315.24,1568.24,1713.73,58.1972,None
1,Infamous,Papita,2,Galaxy Battles II: Emerging Worlds,7,"$20,000",2018-01-19,2018-01-21,1116.27,1172.69,...,1040.73,1061.77,898.102,1686.2,None,315.24,1568.24,1713.73,58.1972,None
2,Infamous,StingeR,3,Galaxy Battles II: Emerging Worlds,7,"$20,000",2018-01-19,2018-01-21,1116.27,1172.69,...,1040.73,1061.77,898.102,1686.2,None,315.24,1568.24,1713.73,58.1972,None
3,Infamous,Scofield,4,Galaxy Battles II: Emerging Worlds,7,"$20,000",2018-01-19,2018-01-21,1116.27,1172.69,...,1040.73,1061.77,898.102,1686.2,None,315.24,1568.24,1713.73,58.1972,None
4,Infamous,Masoku,5,Galaxy Battles II: Emerging Worlds,7,"$20,000",2018-01-19,2018-01-21,1116.27,1172.69,...,1040.73,1061.77,898.102,1686.2,None,315.24,1568.24,1713.73,58.1972,None


In [25]:
startelo

{'elo32': {'current': 1115.4989282630306,
  'sevenDayAgo': 1177.849212773379,
  'sevenDayAvg': 1149.0923960262821,
  'thirtyDayAgo': 1066.920186625756,
  'thirtyDayAvg': 1098.3618020045906},
 'elo64': {'current': 1147.7032625271163,
  'sevenDayAgo': 1302.4662710310354,
  'sevenDayAvg': 1232.35508944966,
  'thirtyDayAgo': 1100.451316100167,
  'thirtyDayAvg': 1156.070769678102},
 'eloRatingDate': 1516312800000,
 'glicko': {'mu': 2030.1074388193833,
  'phi': None,
  'rating': 1787.1707956556083,
  'ratingSevenDaysAgo': 1887.6947139307895,
  'sigma': 97.17465726550999},
 'glicko2': {'mu': 1909.792420668949,
  'phi': 43.14249739161507,
  'rating': 1801.9361771899114,
  'ratingSevenDaysAgo': 1820.3766333561912,
  'sigma': None},
 'glickoRatingDate': 1515967200000,
 'lossesLastMonth': None,
 'teamName': 'Mineski.亿鼎博',
 'valveId': 543897,
 'winsLastMonth': None}

In [27]:
endelolist

[{'elo32': {'current': 1154.3439093947534,
   'sevenDayAgo': 1161.9598747964876,
   'sevenDayAvg': 1158.1256913101236,
   'thirtyDayAgo': 1189.7307806184972,
   'thirtyDayAvg': 1171.4493321351829},
  'elo64': {'current': 1214.953876129925,
   'sevenDayAgo': 1225.5605874021342,
   'sevenDayAvg': 1220.220742082554,
   'thirtyDayAgo': 1264.2369684364662,
   'thirtyDayAvg': 1238.7765001344274},
  'eloRatingDate': 1516485600000,
  'glicko': {'mu': 2185.1768709209837,
   'phi': None,
   'rating': 1623.5106802779346,
   'ratingSevenDaysAgo': 1660.3255493912825,
   'sigma': 224.66647625721959},
  'glicko2': {'mu': 2073.830505603768,
   'phi': 49.64443748393158,
   'rating': 1949.719411893939,
   'ratingSevenDaysAgo': 1952.5037121177882,
   'sigma': None},
  'glickoRatingDate': 1515967200000,
  'lossesLastMonth': None,
  'teamName': 'Team Liquid',
  'valveId': 2163,
  'winsLastMonth': None},
 {'elo32': {'current': 922.8130208244046,
   'sevenDayAgo': 919.0042967537827,
   'sevenDayAvg': 920.921

In [23]:
df[['team', 'tour', 'startglickorating', 'endglickorating', 'start', 'end']]

,team,tour,startglickorating,endglickorating,start,end
0,Infamous,Galaxy Battles II: Emerging Worlds,1646.25,898.102,2018-01-19,2018-01-21
1,Infamous,Galaxy Battles II: Emerging Worlds,1646.25,898.102,2018-01-19,2018-01-21
2,Infamous,Galaxy Battles II: Emerging Worlds,1646.25,898.102,2018-01-19,2018-01-21
3,Infamous,Galaxy Battles II: Emerging Worlds,1646.25,898.102,2018-01-19,2018-01-21
4,Infamous,Galaxy Battles II: Emerging Worlds,1646.25,898.102,2018-01-19,2018-01-21
5,OG,Galaxy Battles II: Emerging Worlds,1838.99,1838.99,2018-01-19,2018-01-21
6,OG,Galaxy Battles II: Emerging Worlds,1838.99,1838.99,2018-01-19,2018-01-21
7,OG,Galaxy Battles II: Emerging Worlds,1838.99,1838.99,2018-01-19,2018-01-21
8,OG,Galaxy Battles II: Emerging Worlds,1838.99,1838.99,2018-01-19,2018-01-21
9,OG,Galaxy Battles II: Emerging Worlds,1838.99,1838.99,2018-01-19,2018-01-21
